# Preliminary items

This is based largely on [the DLC User Guide](https://github.com/DeepLabCut/DeepLabCut/blob/master/docs/standardDeepLabCut_UserGuide.md), which provides more detail than the text offered here.

We've assumed here that DLC has been installed on your system. We'll further assume that you are acquiring 2D data from videos that include a single animal.

It is also critical that you install and an environment for DLC and that you run this code from within the environment. See the [installation instructions](https://github.com/DeepLabCut/DeepLabCut/blob/master/docs/installation.md) for details.

As explained in the user guide, it's a good idea to trim the duration, crop the frames, compress the video a bit, and downsample the resolution of your videos to have DLC run more efficiently. See our [dlc_preprocess](dlc_preprocess.ipynb) notebook for our own command-line tricks to trim down your movies. [Mathis and Warren (2018)](https://www.biorxiv.org/content/10.1101/457242v1.abstract) details the effects of video compression and other factors on the accuracy and speed of DLC.

You'll want to adjust the parameters below for your particular computer and project.

In [ ]:
# Import essential packages
import deeplabcut as dlc
import os
from sources import dlc_help as dlch

# Customize these titles for your project 
projName            = 'geotaxis'
investigatorName    = 'mmchenry'

# Path to project directory (where dlc data will be stored)
dataPath = '/Users/mmchenry/Documents/Projects/geotaxis/dlc'

# Place all your videos for training and analysis in a directory and specify the path to that directory here
videoPath   = '/Users/mmchenry/Documents/Projects/geotaxis/Videos'

# Extension to video files
vidFileExt = 'mp4'

# Create a new project
Run the following line of code to initialize a new project. Skip this section if the project was previously created.

In [ ]:
# Generate list of video files in trainingVideoPath directory
pathList = dlch.list_paths(videoPath, vidFileExt)
# print(pathList) # Uncomment to display list of video files

# dlc command to create the project
dlc.create_new_project(projName, investigatorName, pathList, dataPath, copy_videos=False, multianimal=False)

# Project path
If you just created a new project, find the directories and files created in the project directory at dataPath and open config.yaml (within the dataPath directory) and copy and paste the project_path into the cell below. 

For either new or old projects, excute the cell below to define the project path and location of config.yaml for subsequent cells.

In [ ]:
project_path    = '/Users/mmchenry/Documents/Projects/geotaxis/dlc/geotaxis-mmchenry-2022-05-20' 

# Path to config file
config_path = project_path + os.path.sep + 'config.yaml'

# Designate your list of landmarks

The config.yaml file lists key items for your project (see [this page](https://camo.githubusercontent.com/6a5086ab4a01f9eaf41e6972d86785faed0806c69a83d68a5f8b981f0cae4965/68747470733a2f2f737461746963312e73717561726573706163652e636f6d2f7374617469632f3537663664353163396637343536366635356563663237312f742f3563343066343132346437613963306232636536353163312f313534373736303731363239382f426f78312d30312e706e673f666f726d61743d3130303077) for the full glossary of parameters), including a listing of the landmarks that will be tracked (under the 'bodyparts' heading). 

Delete the entries of '- bodypart1', '- bodypart2' , etc. and key in your own landmarks (e.g., '- nose', '- eye', '- tail').

If you want to include lines that link together these landmarks in the visualization tools later, then you can specify linked points under 'skeleton:', for example a link between the nose and eye would be specified like this:

    - - nose
      - eye

There are a variety of other visualization parameters that you can tweak to vary the display of landmarks.

There are other parameters that will come up as we proceed through the workflow, so keep track of the location of your config.yaml file.

# Extract frames
This section copies frames from your videos for you to annotate with landmark positions.

We have configured the 'extract_frames' function with the defaults. 

Here, we are using the 'kmeans' algorithm to select and downsample frames. This neglects color information, so you would want to enter algo='uniform' if you think color is helpful (though this takes more computational time). Also, you will want to configure things differently if you are doing a supplemental round of training.

It is possible to manually select frames and to exert more control through other parameters, as explained in the user guide. 

Frame extraction might take some time to complete. Copies of the frames are save to directories named after each video, in the 'labeled-data' directory. Browse to those directories to see the frames that will be used for the training.

As a general rule of thumb 100 - 200 labeled frames may be sufficient to train your network, although it never hurts to use more frames. In config.yaml, the 'numframes2paick' parameter sets the number of frames extracted from each video in your project. So, you might adjust that number based on how many videos you have in your project to aim for ~200 frames.


<!-- The choice of which training network to use is another factor. The different options are described [here](https://deeplabcut.github.io/DeepLabCut/docs/recipes/nn.html). The default (ResNet-50) is a good place to start, if you have an expensive GPU and are not sure if your videos are going to offer a big challenge. The MobileNets are worth considering if you're running DLC on a CPU. You can try different networks later. -->

<!-- There is the ability to specify a region-of-interest in the videos, but we haven't had much luck getting that to work and it's not a bad idea to crop the videos ahead of runnign DLC to reduce their file size anyway.  DLC is much more time-efficient with videos of smaller file size. -->

<!-- 'TrainingFraction' is the (number of training videos) / (total number of videos), so you might adjust this based on how many training videos you want to work through. --> 

<!-- On an initial training, shoot for 100 - 200 frames for training. If 'extract_frames' reports the number of frames being extracted outside that range, adjust the 'TrainingFraction' and 'numframes2pick' parameters in config.yaml, delete the contents of the 'labeled-data' directory, and run it again. -->

In [ ]:
# Run command to extract frames for landmark tracking
dlc.extract_frames(config_path, mode='automatic', algo='kmeans', userfeedback=False, crop=False)

# Label frames

Here, we're using the parameter defaults, which assume that we're not training for 3D experiments or tracking multiple animals.

Try launching the GUI for landmark selection by running the cell below.

In [3]:
dlc.label_frames(config_path)

UsageError: Line magic function `%dlc` not found.


On my Mac, this did not work due to trouble with GUIs from a jupyter notebooks. I'm sure there's a fix for this, but I just ran the command outside of this notebook with the following steps:

1. Launch a Terminal window
1. Launch a verson of python that can handle GUIs in the correct environment with the following commands:

        conda activate DEEPLABCUT_M1; pythonw

1. Define config_path, something like this:

        config_path = '/Users/mmchenry/Documents/Projects/geotaxis/dlc/geotaxis-mmchenry-2022-05-20/config.yaml'

1. Import deeplabcut

        import deeplabcut as dlc

1. Then, run the command:

        dlc.label_frames(config_path)

I recommend reading the "(D) Label Frames" section of [the DLC User Guide](https://github.com/DeepLabCut/DeepLabCut/blob/master/docs/standardDeepLabCut_UserGuide.md) to get oriented on how the GUI works. The general idea is to pick off the coordinates of landmarks among all extracted frames from your videos. Skip on selecting coordinates for any landmarks that are not visible.

The coordinates are stored in each directory that holds the extracted frames, within 'labeled-data' in two files (csv and h5) that start with 'CollectedData'. If you load a set of images using label_frames, it is the data in the h5 file that is plotted on the frames, not the csv version. So, I suspect that the csv version is added there to offer a human-readable version of the data that is not used later in the training.